In [1]:
from __future__ import annotations
import math
import random
from enum import Enum, auto
from dataclasses import dataclass, field
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


In [2]:

# if GPU is to be used
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "cpu"
)

@dataclass
class HyperParams:
    batch_size: int = 32
    gamma: float = 0.99
    eps_start: float = 0.9
    eps_end: float = 0.01
    eps_decay: int = 2500
    tau: float = 0.005
    lr: float = 3e-4
    device: torch.device = field(default_factory=lambda: device)

In [3]:
class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [4]:
class StepResult(Enum):
    MOVED = -0.1
    HIT_WALL = -1.0
    KEY_PICKED = 2.0
    DOOR_OPENED = 1.0
    DOOR_BLOCKED = -1.0
    GOAL_REACHED = 10.0
    INVALID = 0.0  # fallback or unused

In [5]:
def compute_reward(result: StepResult) -> float:
    return result.value

In [6]:
class Actions(Enum):
    """
    Enum for actions in the dungeon environment.
    """
    MOVE_UP = 0
    MOVE_DOWN = 1
    MOVE_LEFT = 2
    MOVE_RIGHT = 3
    ATTACK = 4
    PICKUP_ITEM = 5
    USE_ITEM = 6
    OPEN_INVENTORY = 7
    QUIT = 8

In [7]:
class DQNAgent:
    def __init__(self, policy_net, hp):
        self.policy_net = policy_net
        self.hp = hp
        self.steps_done = 0

    def select_action(self, state, action_space):
        sample = random.random()
        eps_threshold = self.hp.EPS_END + (self.hp.EPS_START - self.hp.EPS_END) * \
            math.exp(-1. * self.steps_done / self.hp.EPS_DECAY)
        self.steps_done += 1
        if sample > eps_threshold:
            with torch.no_grad():
                # t.max(1) will return the largest column value of each row.
                # second column on max result is index of where max element was
                # found, so we pick action with the larger expected reward.
                return self.policy_net(state).max(1).indices.view(1, 1)
        else:
            return torch.tensor([[random.choice(action_space)]], device=self.hp.device, dtype=torch.long)    



In [8]:
class GridCellValues(Enum):
    EMPTY=0
    WALL=auto()
    VISITED=auto()
    GOAL=auto()
    PATH=auto()   
    START=auto()

In [ ]:
class DungeonEnv:
    def __init__(self, rows=5, cols=5, wall_prob=0.3, seed=None, device=torch.device("cpu")):
        self.action_space = [a.value for a in Actions if a.value < Actions.ATTACK.value]
        self.observation_space = 2
        self.rows = rows
        self.cols = cols
        self.wall_prob = wall_prob
        self.rng = np.random.RandomState(seed)
        self.grid = None
        self.start_pos = None
        self.goal_pos = None
        self.agent_pos = None
        self.device = device
        self.reset()
     
    def reset(self):
        self.grid = self.rng.choice([GridCellValues.EMPTY, GridCellValues.WALL],
                                     (self.rows, self.cols), 
                                     p=[1 - self.wall_prob, self.wall_prob]) 
        # Find all empty cells *before* placing start/goal
        empty_cells = list(zip(*np.where(self.grid == GridCellValues.EMPTY)))
        # Pick goal position
        self.goal_pos = self.rng.choice(len(empty_cells))
        goal_cell = empty_cells[self.goal_pos]

        # Remove goal cell from list before picking start
        empty_cells.pop(self.goal_pos)
        start_cell = empty_cells[self.rng.choice(len(empty_cells))]

        # Set in grid
        self.grid[goal_cell] = GridCellValues.GOAL
        self.grid[start_cell] = GridCellValues.START
        self.start_pos = start_cell
        self.agent_pos = start_cell 
        # Return state as [1, 2] tensor
        row, col = self.agent_pos
        state = torch.tensor([[row, col]], device=self.device, dtype=torch.float32)  # shape: [1, 2]
        return state


    def step(self, action: int):
        """
        Apply an action in the grid and return the next state, reward, and done flag.

        Args:
            action (int): Integer corresponding to an Actions enum value.

        Returns:
            new_state (torch.Tensor): The new position as a 2D int tensor [row, col]
            reward (float): The reward for this action
            done (bool): True if the episode has ended
            info (dict): Optional debug information
        """
        try:
            action_enum = Actions(action)
        except ValueError:
            # Invalid enum value passed in
            reward = StepResult.INVALID.value
            done = False
            new_state = torch.tensor(self.agent_pos, device=self.device, dtype=torch.int32)
            return new_state, reward, done, {"invalid_action": action}

        row, col = self.agent_pos
        new_row, new_col = row, col

        # Apply movement
        match action_enum:
            case Actions.MOVE_UP:
                new_row -= 1
            case Actions.MOVE_DOWN:
                new_row += 1
            case Actions.MOVE_LEFT:
                new_col -= 1
            case Actions.MOVE_RIGHT:
                new_col += 1
            case Actions.QUIT:
                reward = 0.0
                done = True
                new_state = torch.tensor([row, col], device=self.device, dtype=torch.int32)
                return new_state, reward, done, {"quit": True}
            case _:
                reward = StepResult.INVALID.value
                done = False
                new_state = torch.tensor([row, col], device=self.device, dtype=torch.int32)
                return new_state, reward, done, {"unsupported_action": action_enum.name}

        # Check bounds
        if not (0 <= new_row < self.rows and 0 <= new_col < self.cols):
            reward = StepResult.HIT_WALL.value
            done = False
            new_state = torch.tensor([row, col], device=self.device, dtype=torch.int32)
            return new_state, reward, done, {}

        cell_value = self.grid[new_row, new_col]

        match cell_value:
            case GridCellValues.WALL:
                reward = StepResult.HIT_WALL.value
                done = False
                new_state = torch.tensor([row, col], device=self.device, dtype=torch.int32)

            case GridCellValues.GOAL:
                self.agent_pos = (new_row, new_col)
                reward = StepResult.GOAL_REACHED.value
                done = True
                new_state = torch.tensor([new_row, new_col], device=self.device, dtype=torch.int32)

            case _:
                self.agent_pos = (new_row, new_col)
                reward = StepResult.MOVED.value
                done = False
                new_state = torch.tensor([new_row, new_col], device=self.device, dtype=torch.int32)

        return new_state, reward, done, {}


In [10]:

# Define a named tuple to represent a transition in the environment
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory:

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [11]:
def optimize_model(hp: HyperParams, policy_net, target_net, memory, optimizer):
    if len(memory) < hp.batch_size:
        return
    transitions = memory.sample(hp.batch_size)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1).values
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(hp.batch_size, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * hp.gamma) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

In [13]:

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()




# To ensure reproducibility during training, you can fix the random seeds
# by uncommenting the lines below. This makes the results consistent across
# runs, which is helpful for debugging or comparing different approaches.
#
# That said, allowing randomness can be beneficial in practice, as it lets
# the model explore different training trajectories.


# seed = 42
# random.seed(seed)
# torch.manual_seed(seed)
# env.reset(seed=seed)
# env.action_space.seed(seed)
# env.observation_space.seed(seed)
# if torch.cuda.is_available():
#     torch.cuda.manual_seed(seed

In [ ]:
env = DungeonEnv()
hp = HyperParams()
n_actions = len(env.action_space)
# Get the number of state observations
state = env.reset()
n_observations = state.size
policy_net = DQN(n_observations, n_actions).to(hp.device)
target_net = DQN(n_observations, n_actions).to(hp.device)
target_net.load_state_dict(policy_net.state_dict())

dqn_agent = DQNAgent(policy_net=policy_net, hp=hp)
optimizer = optim.AdamW(policy_net.parameters(), lr=hp.lr, amsgrad=True)
memory = ReplayMemory(1000)

action = dqn_agent.select_action(state, env.action_space)
next_state, reward, done, info = env.step(action)
# Convert next_state to a batch of size 1
next_state = next_state.unsqueeze(0)
reward = torch.tensor([[reward]], device=hp.device, dtype=torch.float32)
action = torch.tensor([[action]], device=hp.device, dtype=torch.int64)